In [ ]:
from cracknuts.cracker import CrackerG1
from cracknuts.jupyter.cracker_g1_panel import CrackerG1Panel

In [ ]:
cracker = CrackerG1('192.168.0.21')

In [ ]:
cracker.connect(force_update_bin=True, force_write_default_config=True)

In [ ]:
import random
import time
from cracknuts.cracker import serial


cmd_set_aes_enc_key = "01 00 00 00 00 00 00 10"
cmd_aes_enc = "01 02 00 00 00 00 00 10"

aes_key = "11 22 33 44 55 66 77 88 99 00 aa bb cc dd ee ff"
aes_data_len = 16

sample_length = 20000

def init():
    cracker.nut_voltage_enable()
    cracker.nut_voltage(3.3)
    cracker.nut_clock_enable()
    cracker.nut_clock_freq('8M')
    cracker.uart_io_enable()
    cracker.osc_sample_clock('48m')
    cracker.osc_sample_length(sample_length)
    cracker.osc_trigger_source('N')
    cracker.osc_analog_gain('B', 10)
    cracker.osc_trigger_level(0)
    cracker.osc_trigger_mode('E')
    cracker.osc_trigger_edge('U')
    cracker.uart_config(baudrate=serial.Baudrate.BAUDRATE_115200, bytesize=serial.Bytesize.EIGHTBITS, parity=serial.Parity.PARITY_NONE, stopbits=serial.Stopbits.STOPBITS_ONE)

    time.sleep(2)
    cmd = cmd_set_aes_enc_key + aes_key
    status, ret = cracker.uart_transmit_receive(cmd, timeout=1000, rx_count=6)

def do(count):
    plaintext_data = random.randbytes(aes_data_len)
    tx_data = bytes.fromhex(cmd_aes_enc.replace(' ', '')) + plaintext_data
    # cracker.glitch_vcc_arm()
    status, ret = cracker.uart_transmit_receive(tx_data, rx_count= 6 + aes_data_len, is_trigger=True)
    return {
        "plaintext": plaintext_data,
        "ciphertext": ret[-aes_data_len:],
        "key": bytes.fromhex(aes_key),
        "glitch_status": 1
    }


def finish():
    ...
    # print('optional behavior')



p = CrackerG1Panel.builder()\
    .with_cracker(cracker)\
    .with_acquisition(init=init, do=do, finish=finish)\
    .build()

p

In [ ]:
# cracker.glitch_gnd_normal(0.1)
# cracker.glitch_vcc_normal(3.5)

cracker.glitch_vcc_config(level=1.5, wait=100, delay=100, repeat=1, count=200)


In [ ]:
cracker.glitch_vcc_force()

In [ ]:
p.acquisition.shadow_trace_count

In [ ]:
from cracknuts.jupyter.glitch_test_result_analysis_panel import GlitchTestResultAnalysisPanel

ap = GlitchTestResultAnalysisPanel(r"D:\project\cracknuts\tests\dataset\20251016170714.db")
ap